# Imports

In [1]:
import pandas as pd
import os 
current_folder = os.getcwd()
print(current_folder)
parent_folder = os.path.dirname(current_folder)
os.chdir(parent_folder)

/home/araceli/code/energy-analysis-and-predictions/notebooks


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from utils.parameters import ENERGY_COMPANIES_FOLDER, ENERGY_COMPANY_SECTOR

from load_data.load_energy_prices import load_energy_prices_data, \
                            load_energy_prices_from_all_files         

from predict_data.create_train_test_data import get_scaled_X_y_train_and_test_data
from predict_data.create_simple_train_test_data import get_X_y_scaled_and_splited                       
from predict_data.load_gru_model import load_gru_model          
from predict_data.model_methods import compile_model, train_model,evaluate_model, predict_model                                          

from visualize_data.plot_price_with_interactive_timestamps import plot_price_with_interactive_timestamps   
from visualize_data.plot_model_history import plot_history
from visualize_data.plot_predict_vs_test import plot_predict_vs_test

2023-05-21 16:36:04.710497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# COMPANY SECTORS

In [53]:
energy_sectors = pd.read_csv(ENERGY_COMPANY_SECTOR ,delimiter = ",", encoding = "utf-8") 
energy_sectors.head()

,Name,Place,Energy_Source,Rated_capacity
0,Abfallentsorgungszentrum Asdonkshof,Kamp-Lintfort,Abfall,20.8 MW
1,Abfallheizkraftwerk Neunkirchen,Neunkirchen,Abfall,11.6 MW
2,AMK - Abfallentsorgungsgesellschaft des Maerki...,Iserlohn,Abfall,12.6 MW
3,AVA GmbH,Augsburg,Abfall,10 MW
4,AVA Velsen,Saarbruecken,Abfall,16 MW


In [55]:
energy_sectors['Energy_Source'] = energy_sectors['Energy_Source'].map(lambda x: str(x)[:-1])

In [56]:
energy_sector_types = energy_sectors['Energy_Source'].value_counts()
energy_sector_types

Erdgas                                    269
Laufwasser                                104
Wind (Onshore)                             90
Steinkohle                                 80
Photovoltaik                               63
Biomasse                                   56
na                                         55
Abfall                                     45
Braunkohle                                 28
Pumpspeicher                               28
Wind (Offshore)                            28
Sonstige konventionelle Energietraeger     25
Mineraloelprodukte                         23
Speicherwasser (ohne Pumpspeicher)         21
Batteriespeicher                           15
Kernenergie                                 8
Steinkohle - Heizoe                         1
Biomass                                     1
Waerme                                      1
Erdgas – Heizoe                             1
Name: Energy_Source, dtype: int64

In [57]:
sectors = list(energy_sector_types.index)
#sectors = [sector[:-1] for sector in sectors]
sectors.sort()
sectors

['Abfall',
 'Batteriespeicher',
 'Biomass',
 'Biomasse',
 'Braunkohle',
 'Erdgas',
 'Erdgas – Heizoe',
 'Kernenergie',
 'Laufwasser',
 'Mineraloelprodukte',
 'Photovoltaik',
 'Pumpspeicher',
 'Sonstige konventionelle Energietraeger',
 'Speicherwasser (ohne Pumpspeicher)',
 'Steinkohle',
 'Steinkohle - Heizoe',
 'Waerme',
 'Wind (Offshore)',
 'Wind (Onshore)',
 'na']

In [64]:
dict_companies = {sector:[] for sector in sectors}
dict_companies

{'Abfall': [],
 'Batteriespeicher': [],
 'Biomass': [],
 'Biomasse': [],
 'Braunkohle': [],
 'Erdgas': [],
 'Erdgas – Heizoe': [],
 'Kernenergie': [],
 'Laufwasser': [],
 'Mineraloelprodukte': [],
 'Photovoltaik': [],
 'Pumpspeicher': [],
 'Sonstige konventionelle Energietraeger': [],
 'Speicherwasser (ohne Pumpspeicher)': [],
 'Steinkohle': [],
 'Steinkohle - Heizoe': [],
 'Waerme': [],
 'Wind (Offshore)': [],
 'Wind (Onshore)': [],
 'na': []}

# Power Plant data

In [4]:
#foldername = os.path.join(ENERGY_COMPANIES_FOLDER,'power-plant-data-2022-Jan-to-Dec' )
power_plant_list = os.listdir(ENERGY_COMPANIES_FOLDER)
power_plant_list.sort()
print('len(power_plant_list)=', len(power_plant_list))


len(power_plant_list)= 226


In [13]:
power_plant_list[:10]

['Abwinden-Asten_202201010000_202212312359_Tag_1.csv',
 'Abwinden-Asten_202201010000_202212312359_Tag_1.csv:Zone.Identifier',
 'Altenw_rth_202201010000_202212312359_Tag_3.csv',
 'Altenw_rth_202201010000_202212312359_Tag_3.csv:Zone.Identifier',
 'Bergkamen_202201010000_202212312359_Tag_6.csv',
 'Bergkamen_202201010000_202212312359_Tag_6.csv:Zone.Identifier',
 'Boxberg_202201010000_202212312359_Tag_8.csv',
 'Boxberg_202201010000_202212312359_Tag_8.csv:Zone.Identifier',
 'Braunkohlekraftwerk_Lippendorf_202201010000_202212312359_Tag_9.csv',
 'Braunkohlekraftwerk_Lippendorf_202201010000_202212312359_Tag_9.csv:Zone.Identifier']

In [15]:
for file in power_plant_list:
    if 'Identifier' in file:
        power_plant_list.remove(file)
        
len(power_plant_list)

113

In [86]:
filename = os.path.join(ENERGY_COMPANIES_FOLDER, power_plant_list[0])
print(filename)
df = pd.read_csv(filename, delimiter=';', decimal='.')
df.head()

/home/araceli/code/energy-analysis-and-predictions/raw_data/energy-companies/power-plant-data-2022-Jan-to-Dec/Abwinden-Asten_202201010000_202212312359_Tag_1.csv


,Datum,Anfang,Ende,Generation_DE [MW] Berechnete Auflösungen
0,01.01.2022,00:00,00:00,3.906
1,02.01.2022,00:00,00:00,3.813
2,03.01.2022,00:00,00:00,3.479
3,04.01.2022,00:00,00:00,3.488
4,05.01.2022,00:00,00:00,3.667


In [88]:
print(len(df.columns))
df.columns


4


Index(['Datum', 'Anfang', 'Ende',
       'Generation_DE  [MW] Berechnete Auflösungen'],
      dtype='object')

In [80]:
filename = os.path.join(ENERGY_COMPANIES_FOLDER, power_plant_list[4])
print(filename)
df = pd.read_csv(filename, delimiter=';')
df.head()

/home/araceli/code/energy-analysis-and-predictions/raw_data/energy-companies/power-plant-data-2022-Jan-to-Dec/Braunkohlekraftwerk_Lippendorf_202201010000_202212312359_Tag_9.csv


,Datum,Anfang,Ende,Generation_DE R [MW] Berechnete Auflösungen,Generation_DE LIP S [MW] Berechnete Auflösungen
0,01.01.2022,00:00,00:00,9.438,15.926
1,02.01.2022,00:00,00:00,0,10.678
2,03.01.2022,00:00,00:00,13.336,17.192
3,04.01.2022,00:00,00:00,19.956,20.004
4,05.01.2022,00:00,00:00,16.153,16.118


In [81]:
cols =  list(df.columns[3:])
for col in cols:
    df[col] = df[col].str.replace('.', '')
    df[col] = df[col].str.replace(',', '.')
    df[col] = df[col].replace('-', np.nan)
    df[col] = df[col].astype('float32')
df.head()

/tmp/ipykernel_2375/2754829033.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.', '')


,Datum,Anfang,Ende,Generation_DE R [MW] Berechnete Auflösungen,Generation_DE LIP S [MW] Berechnete Auflösungen
0,01.01.2022,00:00,00:00,9438.0,15926.0
1,02.01.2022,00:00,00:00,0.0,10678.0
2,03.01.2022,00:00,00:00,13336.0,17192.0
3,04.01.2022,00:00,00:00,19956.0,20004.0
4,05.01.2022,00:00,00:00,16153.0,16118.0


In [82]:
#nan_val = 0
from sklearn.impute import SimpleImputer

for col in cols:
    nan_val = (df[col].isna().value_counts()*100/df.shape[0])
    print('nan values % = ', nan_val[1])
    if nan_val[1] < 30:
        imputer = SimpleImputer(strategy="mean") 
        df[col] = imputer.fit_transform(df[[col]])

nan values % =  1.095890410958904
nan values % =  1.095890410958904


In [83]:
if len(df.columns)>4:
    print(len(df.columns))
    
    df['avg energy [MW]'] = df[cols].mean(axis=1)
    df[cols]
df.head()

5


,Datum,Anfang,Ende,Generation_DE R [MW] Berechnete Auflösungen,Generation_DE LIP S [MW] Berechnete Auflösungen,avg energy [MW]
0,01.01.2022,00:00,00:00,9438.0,15926.0,12682.0
1,02.01.2022,00:00,00:00,0.0,10678.0,5339.0
2,03.01.2022,00:00,00:00,13336.0,17192.0,15264.0
3,04.01.2022,00:00,00:00,19956.0,20004.0,19980.0
4,05.01.2022,00:00,00:00,16153.0,16118.0,16135.5


In [ ]:
test =  pd.read_csv('SMARD/power-plant-data-2022-Jan-to-Dec/Amrumbank_West_202201010000_202212312359_Tag_4.csv',delimiter=';', decimal=',') 
test

In [ ]:
test.shape

(1, 4)

In [ ]:
test2 = pd.read_csv('SMARD/power-plant-data-2022-Jan-to-Dec/Pumpspeicherkraftwerk_Vianden_202201010000_202212312359_Tag_137.csv', delimiter=';', decimal=',') 
test2.head()

In [ ]:
num_cols = test2.columns[3:len(test2)]

In [ ]:
def convert_str_to_float(text):
    if text == '-':
        return 0
    else:
        text=text.replace(',','.')
        return float(text)

In [ ]:
test2['Generation_DE Maschine 9 [MW] Berechnete Auflösungen']=test2['Generation_DE Maschine 9 [MW] Berechnete Auflösungen'].apply(from_str_to_float)

In [ ]:
test2['Generation_DE Maschine 9 [MW] Berechnete Auflösungen'].head()

0    148.0
1      0.0
2    257.0
3    210.0
4      0.0
Name: Generation_DE Maschine 9 [MW] Berechnete Auflösungen, dtype: float64

In [ ]:
for col in num_cols:
    test2[col] = test2[col].apply(convert_str_to_float)  

In [ ]:
test2.dtypes

Datum                                                     object
Anfang                                                    object
Ende                                                      object
Generation_DE Maschine 9 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 8 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 2 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 3 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 4 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 7 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 1 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 5 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 6 [MW] Berechnete Auflösungen     float64
Generation_DE Maschine 10 [MW] Berechnete Auflösungen    float64
Generation_DE Maschine 11 [MW] Berechnete Auflösungen    float64
dtype: object

In [ ]:
test2['Generation_DE [MW] Berechnete Auflösungen'] = test2.sum(numeric_only = True, axis =1)
test2.head()

In [ ]:
148.0 + 176.0+ 0.0+ 1.475+ 765.000+ 232.0+ 1.476+ 460.0	+ 391.0+ 0.0+ 292.0	

2466.951

In [ ]:
type(test2['Generation_DE Maschine 9 [MW] Berechnete Auflösungen'][177])

str

In [ ]:
test2['Generation_DE Maschine 9 [MW] Berechnete Auflösungen'][178]

'-'

In [ ]:
float(test2['Generation_DE Maschine 9 [MW] Berechnete Auflösungen'][177])

230.0

In [ ]:
# Take first company
file1 = power_plant_list[0].split('_')[0]
col = file1.split('_')[0]
folder = path + '/' + power_plant_list[0]
power_plant = pd.read_csv(folder, delimiter=';', decimal=',')
power_plant.columns = ['Date', 'Start', 'End', col]
power_plant.head()

,Date,Start,End,Abwinden-Asten
0,01.01.2022,00:00,00:00,3.906
1,02.01.2022,00:00,00:00,3.813
2,03.01.2022,00:00,00:00,3.479
3,04.01.2022,00:00,00:00,3.488
4,05.01.2022,00:00,00:00,3.667


In [ ]:
power_plant_list.pop(1)
power_plant_list[0:5]

['Abwinden-Asten_202201010000_202212312359_Tag_1.csv',
 'Ahrensfelde_202201010000_202212312359_Tag_2.csv',
 'Ahrensfelde_202201010000_202212312359_Tag_2.csv:Zone.Identifier',
 'Altenw_rth_202201010000_202212312359_Tag_3.csv',
 'Altenw_rth_202201010000_202212312359_Tag_3.csv:Zone.Identifier']

In [ ]:
for file in power_plant_list:
    col = file.split('_')[0]
    folder = path + '/' + file
    df = pd.read_csv(folder, delimiter=';', decimal=',')
    
    if df.shape[0] > 1:
        if df.shape[2] > 4:
            num_cols =  df.columns[3:len(test2)]
            for col in num_cols:
                test2[col] = test2[col].apply(convert_str_to_float)  
            df['Generation_DE [MW] Berechnete Auflösungen'] = df.sum(numeric_only = True, axis =1)
        data_name = 'Generation_DE [MW] Berechnete Auflösungen' 
        for i in df.columns:
            if 'Berechnete Auflösungen' in i:
                data_name = i
        power_plant[col] = df[data_name]
    else:
        print(f'Company {col} data is empty')


,Datum,Anfang,Ende,Generation_DE [MW] Berechnete Auflösungen
0,01.01.2022,00:00,00:00,3.906
1,02.01.2022,00:00,00:00,3.813
2,03.01.2022,00:00,00:00,3.479
3,04.01.2022,00:00,00:00,3.488
4,05.01.2022,00:00,00:00,3.667


In [ ]:
power_plant_dict= {
    'Laufwasser': ['Abwinden-Asten'],
    'Erdgas' : ['Ahrensfelde']    
}
    